#Librerías y paquetes

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163007 sha256=3b91db57914ff1f60064d05939d2b905c75e32176dfc97d92e1870df02d02bed
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import numpy as np
import pandas as pd
import sqlite3

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from ipywidgets import interact ## para análisis interactivo
from sklearn import neighbors ### basado en contenido un solo producto consumido
import joblib
####Paquete para sistemas de recomendación surprise
###Puede generar problemas en instalación local de pyhton. Genera error instalando con pip
#### probar que les funcione para la próxima clase

from surprise import Reader, Dataset
from surprise.model_selection import cross_validate, GridSearchCV
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise.model_selection import train_test_split

In [7]:
# Conectarse a la base de datos
conn = sqlite3.connect('db_moviesF')
cur=conn.cursor()

#Verificar tablas creadas en el script
# Conectarse a la base de datos
conn = sqlite3.connect('db_moviesF')

# Crear un cursor
cursor = conn.cursor()

# Obtener una lista de tablas en la base de datos
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

# Imprimir los nombres de las tablas para verificar la información
for table in tables:
    print(table[0])

In [8]:
# Verificar las columnas de la tabla ratings_final
cursor.execute("PRAGMA table_info(ratings_final)")

columns_ratings_final = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_ratings_final:
    print(column[1], "-", column[2])


user_id - INT
movie_id - INT
rating - REAL


In [9]:
# Verificar las columnas de la tabla ratings
cursor.execute("PRAGMA table_info(ratings)")

columns_ratings = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_ratings:
    print(column[1], "-", column[2])


userId - INTEGER
movieId - INTEGER
rating - REAL
timestamp - INTEGER


In [10]:
# Verificar las columnas de la tabla movies
cursor.execute("PRAGMA table_info(movies)")

columns_movies = cursor.fetchall()

# Imprimir el nombre de la columna y su tipo de datos
for column in columns_movies:
    print(column[1], "-", column[2])


movieId - INTEGER
title - TEXT
genres - TEXT
year - INTEGER


#Sistema de recomendación basado en contenido KNN

##Con base en todo lo visto por el usuario

In [11]:
##### cargar dataframe escalado y con dummies ###
#movies_dum2= joblib.load('salidas_bmovies_dum2.joblib')
movies_dum2=pd.read_csv('/content/movies_dummies.csv')

In [12]:
#### seleccionar usuario para recomendaciones ####
usuarios=pd.read_sql('select distinct (user_id) as user_id from ratings_final',conn)
print (usuarios)
user_id=1 ### para ejemplo manual

     user_id
0          1
1          4
2          6
3          7
4         10
..       ...
373      605
374      606
375      607
376      608
377      610

[378 rows x 1 columns]


In [13]:
def recomendar(user_id=list(usuarios['user_id'].value_counts().index)):

    ###seleccionar solo los ratings del usuario seleccionado
    ratings=pd.read_sql('select *from ratings_final where user_id=:user',conn, params={'user':user_id,})

    ###convertir ratings del usuario a array
    l_movies_w=ratings['movie_id'].to_numpy()

In [55]:
# Cargar los datos de la tabla 'ratings' en un DataFrame
ratings = pd.read_sql('SELECT * FROM ratings', conn)

cursor.execute("DROP TABLE IF EXISTS l_movies_w")

# crear la tabla l_movies_w en la base de datos
consulta_sql = """
CREATE TABLE l_movies_w AS
SELECT movieId AS movie_id FROM ratings;
"""

cursor.execute(consulta_sql)

# Leer los datos de la tabla l_movies_r en un DataFrame de Pandas
l_movies_w = pd.read_sql('SELECT * FROM l_movies_w', conn)


In [56]:
# Cargar los datos de la tabla 'movies' en un DataFrame
movies = pd.read_sql('SELECT * FROM movies', conn)

# Agregar las columnas 'movieId' y 'title' al DataFrame 'movies_dum2'
movies_dum2['movie_id'] = movies['movieId']
movies_dum2['title'] = movies['title']

print(movies_dum2)

       year_sc  Adventure  Animation  Children  Comedy  Fantasy  Romance  \
0     0.988603        1.0        1.0       1.0     1.0      1.0      0.0   
1     0.988603        1.0        0.0       1.0     0.0      1.0      0.0   
2     0.988603        0.0        0.0       0.0     1.0      0.0      1.0   
3     0.988603        0.0        0.0       0.0     1.0      0.0      1.0   
4     0.988603        0.0        0.0       0.0     1.0      0.0      0.0   
...        ...        ...        ...       ...     ...      ...      ...   
9737  0.999504        0.0        1.0       0.0     1.0      1.0      0.0   
9738  0.999504        0.0        1.0       0.0     1.0      1.0      0.0   
9739  0.999504        0.0        0.0       0.0     0.0      0.0      0.0   
9740  1.000000        0.0        1.0       0.0     0.0      0.0      0.0   
9741  0.986620        0.0        0.0       0.0     1.0      0.0      0.0   

      Drama  Action  Crime  ...  Sci-Fi  War  Musical  Documentary  IMAX  \
0       0.0

In [57]:
# Verificar si hay valores duplicados en la columna 'movie_id' de la tabla l_movies_w
duplicates_count = l_movies_w['movie_id'].duplicated().sum()

if duplicates_count == 0:
    print("No hay valores duplicados en la columna 'movie_id' de l_movies_w.")
else:
    print(f"Se encontraron {duplicates_count} valores duplicados en la columna 'movie_id' de l_movies_w.")


Se encontraron 91112 valores duplicados en la columna 'movie_id' de l_movies_w.


In [58]:
# Como hay demasiados valores duplicados, se procede a eliminar valores duplicados en la columna 'movie_id' de la tabla l_movies_w
l_movies_w = l_movies_w.drop_duplicates(subset=['movie_id'])

# Verificar si hay valores duplicados después de la eliminación
duplicates_count_after_removal = l_movies_w['movie_id'].duplicated().sum()

if duplicates_count_after_removal == 0:
    print("Se eliminaron los valores duplicados con éxito.")
else:
    print("Aún quedan valores duplicados después de eliminarlos.")

Se eliminaron los valores duplicados con éxito.


In [59]:
#Verificamos el tamaño del dataframe resultante y aún quedan 9724 registros
print(l_movies_w.shape)

(9724, 1)


In [74]:
# verificamos que tenga la el valor de cruce movie_id
l_movies_w.head(10)

,movie_id
0,1
1,3
2,6
3,47
4,50
5,70
6,101
7,110
8,151
9,157


In [33]:
#verificamos el tipo de dato
l_movies_w.dtypes

movie_id    int64
dtype: object

In [40]:
# verificamos nulos
l_movies_w.isnull().sum()

movie_id    0
dtype: int64

In [75]:
# verificamos que tenga la el valor de cruce movie_id
movies_dum2.head(10)

,year_sc,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),movie_id,title
0,0.988603,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Toy Story (1995)
1,0.988603,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Jumanji (1995)
2,0.988603,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Grumpier Old Men (1995)
3,0.988603,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,Waiting to Exhale (1995)
4,0.988603,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,Father of the Bride Part II (1995)
5,0.988603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6,Heat (1995)
6,0.988603,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,Sabrina (1995)
7,0.988603,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8,Tom and Huck (1995)
8,0.988603,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9,Sudden Death (1995)
9,0.988603,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10,GoldenEye (1995)


In [36]:
#verificamos el tipo de dato
movies_dum2.dtypes

year_sc               float64
Adventure             float64
Animation             float64
Children              float64
Comedy                float64
Fantasy               float64
Romance               float64
Drama                 float64
Action                float64
Crime                 float64
Thriller              float64
Horror                float64
Mystery               float64
Sci-Fi                float64
War                   float64
Musical               float64
Documentary           float64
IMAX                  float64
Western               float64
Film-Noir             float64
(no genres listed)    float64
movie_id                int64
title                  object
dtype: object

In [41]:
# verificamos nulos
movies_dum2.isnull().sum()

year_sc               0
Adventure             0
Animation             0
Children              0
Comedy                0
Fantasy               0
Romance               0
Drama                 0
Action                0
Crime                 0
Thriller              0
Horror                0
Mystery               0
Sci-Fi                0
War                   0
Musical               0
Documentary           0
IMAX                  0
Western               0
Film-Noir             0
(no genres listed)    0
movie_id              0
title                 0
dtype: int64

In [68]:
print(movies_dum2['movie_id'].unique())
print(l_movies_w['movie_id'].unique())

[     1      2      3 ... 193585 193587 193609]
[     1      3      6 ... 160836 163937 163981]


In [70]:
# Verificamos si hay valores duplicados en 'movie_id'
if movies_dum2['movie_id'].duplicated().any():
    print("¡Hay valores duplicados en 'movie_id' en el DataFrame movies_dum2!")
else:
    print("No hay valores duplicados en 'movie_id' en el DataFrame movies_dum2.")


No hay valores duplicados en 'movie_id' en el DataFrame movies_dum2.


In [71]:
# Verificamos si hay valores duplicados en 'movie_id'
if l_movies_w['movie_id'].duplicated().any():
    print("¡Hay valores duplicados en 'movie_id' en el DataFrame movies_dum2!")
else:
    print("No hay valores duplicados en 'movie_id' en el DataFrame movies_dum2.")

No hay valores duplicados en 'movie_id' en el DataFrame movies_dum2.


In [61]:
# Encontrar coincidencias entre movies_dum2 y l_movies_w usando la columna movie_id
movies_w = movies_dum2[movies_dum2['movie_id'].isin(l_movies_w['movie_id'])]
movies_w.head()

,year_sc,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,...,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed),movie_id,title
0,0.988603,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Toy Story (1995)
1,0.988603,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,Jumanji (1995)
2,0.988603,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Grumpier Old Men (1995)
3,0.988603,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4,Waiting to Exhale (1995)
4,0.988603,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,Father of the Bride Part II (1995)


In [78]:
#Verificamos las dimensiones de movies_w
print (movies_w.shape)



(9724, 23)


In [79]:
## eliminar columna nombre y movie_id
movies_w=movies_w.drop(columns=['movie_id','title'])
movies_w["indice"]=1 ### para usar group by y que quede en formato pandas tabla de centroide
##centroide o perfil del usuario
centroide=movies_w.groupby("indice").mean()

In [80]:
#Verificamos el tamaño de centroide
print(centroide)


         year_sc  Adventure  Animation  Children    Comedy   Fantasy  \
indice                                                                 
1       0.986298   0.129782   0.062731  0.068285  0.385952  0.080008   

         Romance     Drama    Action     Crime  ...    Horror   Mystery  \
indice                                          ...                       
1       0.163616  0.447244  0.187988  0.122995  ...  0.100473  0.058926   

          Sci-Fi       War   Musical  Documentary      IMAX   Western  \
indice                                                                  
1       0.100782  0.039181  0.034245     0.045043  0.016248  0.017174   

        Film-Noir  (no genres listed)  
indice                                 
1        0.008741            0.003497  

[1 rows x 21 columns]


In [81]:
 ### filtrar películas no vistas
movies_nw=movies_dum2[~movies_dum2['movie_id'].isin(l_movies_w)]
## eliminar nombre e id
movies_nw=movies_nw.drop(columns=['movie_id','title'])

In [82]:
#Verificamos el tamaño de movies_nw
print (movies_nw.shape)

(9742, 21)


In [83]:
### Entrenar modelo
model=neighbors.NearestNeighbors(n_neighbors=11, metric='cosine')
model.fit(movies_nw)
dist, idlist = model.kneighbors(centroide)


In [84]:
ids=idlist[0] ### queda en un array anidado, para sacarlo
recomend_b=movies.loc[ids][['title','movieId']]
vistas=movies[movies['movieId'].isin(l_movies_w)][['title','movieId']]

In [85]:
return recomend_b

SyntaxError: 'return' outside function (<ipython-input-85-f7b3e3a38a70>, line 1)

In [86]:
recomendar(52853)


print(interact(recomendar))

interactive(children=(Dropdown(description='user_id', options=(1, 404, 415, 414, 413, 412, 411, 410, 409, 408,…

<function recomendar at 0x7dc8418ac940>


##Sistema de recomendación filtro colaborativo

In [87]:
### datos originales en pandas
## knn solo sirve para calificaciones explicitas
ratings=pd.read_sql('select * from ratings_final where rating>0', conn)

In [88]:
####los datos deben ser leídos en un formato especial para surprise
reader = Reader(rating_scale=(0, 10)) ### la escala de la calificación
###las columnas deben estar en orden estándar: user item rating
data   = Dataset.load_from_df(ratings[['user_id','movie_id','rating']], reader)

In [89]:
#####Existen varios modelos
models=[KNNBasic(),KNNWithMeans(),KNNWithZScore(),KNNBaseline()]
results = {}
###knnBasiscs: calcula el rating ponderando por distancia con usuario/Items
###KnnWith means: en la ponderación se resta la media del rating, y al final se suma la media general
####KnnwithZscores: estandariza el rating restando media y dividiendo por desviación
####Knnbaseline: calculan el desvío de cada calificación con respecto al promedio y con base en esos calculan la ponderación


In [90]:
#### for para probar varios modelos ##########
model=models[1]
for model in models:

    CV_scores = cross_validate(model, data, measures=["MAE","RMSE"], cv=5, n_jobs=-1)

    result = pd.DataFrame.from_dict(CV_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_rmse': 'RMSE'})
    results[str(model).split("algorithms.")[1].split("object ")[0]] = result

In [91]:
performance_df = pd.DataFrame.from_dict(results).T
performance_df.sort_values(by='RMSE')

,MAE,RMSE,fit_time,test_time
knns.KNNBaseline,0.626446,0.823525,0.116919,1.908993
knns.KNNWithZScore,0.631700,0.832549,0.133786,1.312306
knns.KNNWithMeans,0.636000,0.833684,0.113178,1.392642
knns.KNNBasic,0.663630,0.873773,0.072123,1.275414


In [92]:
# Se escoge el mejor knn withmeans
param_grid = { 'sim_options' : {'name': ['msd','cosine'], \
                                'min_support': [5,2], \
                                'user_based': [False, True]}
             }
## min support es la cantidad de items o usuarios que necesita para calcular recomendación
## name medidas de distancia

### se afina si es basado en usuario o basado en ítem

In [93]:
gridsearchKNNWithMeans = GridSearchCV(KNNWithMeans, param_grid, measures=['rmse'], \
                                      cv=2, n_jobs=-1)

In [94]:
gridsearchKNNWithMeans.fit(data)

In [95]:
gridsearchKNNWithMeans.best_params["rmse"]
gridsearchKNNWithMeans.best_score["rmse"]
gs_model=gridsearchKNNWithMeans.best_estimator['rmse'] ### mejor estimador de gridsearch

## Entrenar con todos los datos y Realizar predicciones con el modelo afinado

In [96]:
trainset = data.build_full_trainset() ### esta función convierte todos los datos en entrenamiento, las funciones anteriores dividen  en entrenamiento y evaluación
model=gs_model.fit(trainset) ## se reentrena sobre todos los datos posibles (sin dividir)


Computing the msd similarity matrix...
Done computing similarity matrix.


In [97]:
predset = trainset.build_anti_testset() ### crea una tabla con todos los usuarios y las películas que no han visto
#### en la columna de rating pone el promedio de todos los rating, en caso de que no pueda calcularlo para un item-usuario
len(predset)

128594

In [98]:
predictions = gs_model.test(predset) ### función muy pesada, hace las predicciones de rating para todas las películas que no ha visto un usuario
### la funcion test recibe un test set construido con build_test method, o el que genera crosvalidate

In [102]:
####### predicción para una película puntual
model.predict(uid=269397, iid='0446353205',r_ui='') ### uid debía estar en número e isb en comillas

predictions_df = pd.DataFrame(predictions) ### esta tabla se puede llevar a una base donde estarán todas las predicciones
predictions_df.shape
predictions_df.head()
predictions_df['r_ui'].unique() ### promedio de ratings
predictions_df.sort_values(by='est',ascending=False)

,uid,iid,r_ui,est,details
125579,597,318,3.713357,5.235977,"{'actual_k': 40, 'was_impossible': False}"
92535,429,318,3.713357,5.232050,"{'actual_k': 25, 'was_impossible': False}"
109984,523,858,3.713357,5.212990,"{'actual_k': 37, 'was_impossible': False}"
113059,543,318,3.713357,5.210491,"{'actual_k': 38, 'was_impossible': False}"
110017,523,1276,3.713357,5.175731,"{'actual_k': 37, 'was_impossible': False}"
...,...,...,...,...,...
34458,153,1544,3.713357,0.754449,"{'actual_k': 40, 'was_impossible': False}"
34479,153,420,3.713357,0.696163,"{'actual_k': 40, 'was_impossible': False}"
34345,153,432,3.713357,0.675722,"{'actual_k': 40, 'was_impossible': False}"
34364,153,2701,3.713357,0.449150,"{'actual_k': 40, 'was_impossible': False}"


##Función para recomendar las 10 películas con mejores predicciones y llevar a la base de datos para consultar el resto de información

In [101]:
def recomendaciones(user_id,n_recomend=10):

    predictions_userID = predictions_df[predictions_df['uid'] == user_id].\
                    sort_values(by="est", ascending = False).head(n_recomend)

    recomendadas = predictions_userID[['iid','est']]
    recomendadas.to_sql('reco',conn,if_exists="replace")

    recomendadas=pd.read_sql('''select a.*, b.movie_title
                             from reco a left join movies_final b
                             on a.iid=b.movie_id ''', conn)

    return(recomendadas)



recomendaciones(user_id=55490,n_recomend=10)

,index,iid,est,movie_title
